In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
print("All imports OK")

All imports OK


In [5]:
import nltk

nltk.download("wordnet", force=True)
nltk.download("omw-1.4", force=True)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gaugaura\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gaugaura\AppData\Roaming\nltk_data...


True

In [34]:
df = pd.read_csv(filepath_or_buffer="IMDB.csv")
df = df.sample(500)
df.to_csv("data.csv",index=False)
df.head()


,review,sentiment
392,"Okay, I am a fan of the Nightmare series and e...",positive
652,"Anyone who thinks Kool Moe Dee, Carol Alt, and...",negative
267,An excellent documentry. I personally remember...,positive
221,I have been watching movies from i think last ...,negative
673,"So, Steve Irwin. You have to admire a man who ...",negative


In [35]:
df.shape

(500, 2)

In [36]:
# data cleaning 

def lemmatization(text):
    """Lemmatize the text word by word."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def lower_case(text):
    return text.lower()

def remove_numbers(text):
    return "".join(char for char in text if not char.isdigit())


def removing_urls(text):
    """Remove URLs from the text."""
    URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
    return URL_PATTERN.sub('', text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    text = text.replace("؛", "")
    text = re.sub(r"\s+", " ", text).strip()
    return text
    
def normalize_text(df):
    try:
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(remove_numbers)
        df['review'] = df["review"].apply(lower_case)
        df["review"] = df["review"].apply(lemmatization)
        return df
    except Exception as e:
        raise e
    

In [37]:
df = normalize_text(df)
df.head()


,review,sentiment
392,okay i am a fan of the nightmare series and ev...,positive
652,anyone who think kool moe dee carol alt and co...,negative
267,an excellent documentry i personally remember ...,positive
221,i have been watching movie from i think last y...,negative
673,so steve irwin you have to admire a man who is...,negative


In [38]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [39]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
392,okay i am a fan of the nightmare series and ev...,1
652,anyone who think kool moe dee carol alt and co...,0
267,an excellent documentry i personally remember ...,1
221,i have been watching movie from i think last y...,0
673,so steve irwin you have to admire a man who is...,0


In [40]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [49]:
vectorizer= CountVectorizer(max_features=100, stop_words="english")
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [52]:
X.shape


(500, 100)

In [50]:
vectorizer.get_feature_names_out()

array(['acting', 'action', 'actor', 'actually', 'bad', 'best', 'better',
       'big', 'book', 'br', 'cast', 'character', 'come', 'comedy', 'day',
       'did', 'didn', 'director', 'doe', 'doesn', 'don', 'end', 'episode',
       'fact', 'family', 'fan', 'feel', 'film', 'funny', 'girl', 'going',
       'good', 'got', 'great', 'guy', 'ha', 'horror', 'interesting',
       'isn', 'just', 'kid', 'kind', 'know', 'life', 'like', 'line',
       'little', 'long', 'look', 'looking', 'lot', 'love', 'make',
       'making', 'man', 'minute', 'movie', 'music', 'new', 'old',
       'people', 'performance', 'play', 'plot', 'point', 'pretty',
       'quite', 'real', 'really', 'reason', 'right', 'role', 'saw', 'say',
       'scene', 'screen', 'script', 'seen', 'series', 'set', 'star',
       'story', 'tell', 'thing', 'think', 'thought', 'time', 'try', 'tv',
       've', 'version', 'wa', 'want', 'watch', 'watching', 'way', 'woman',
       'work', 'world', 'year'], dtype=object)

In [51]:
X_df = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
X_df.head()

,acting,action,actor,actually,bad,best,better,big,book,br,...,version,wa,want,watch,watching,way,woman,work,world,year
0,0,0,0,0,0,0,0,0,0,6,...,0,6,0,0,0,1,0,0,0,0
1,1,1,1,1,2,0,0,0,0,4,...,0,4,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,3,0,0,0,0,0,...,0,4,0,2,2,0,0,0,0,1
4,1,0,0,0,2,0,0,0,0,6,...,0,8,1,0,0,1,0,0,0,0


In [55]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [59]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/chauhan7gaurav/mlops_capstone.mlflow")
dagshub.init(repo_owner='chauhan7gaurav', repo_name='mlops_capstone', mlflow=True)
mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "chauhan7gaurav/mlops_capstone"

Repository chauhan7gaurav/mlops_capstone initialized!

2026/02/08 17:36:15 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/6e08aa028a0948a094a6d7e8120ec5b5', creation_time=1770552376367, experiment_id='0', last_update_time=1770552376367, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [61]:
import mlflow
import logging
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score , recall_score,f1_score

logging.basicConfig(level=logging.INFO,format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("starting mlflow run   ")

with mlflow.start_run():
    start_time = time.time()
    try:
        logging.info("logging model parameters ")
        mlflow.log_param("vectorizer","Bag of Words")
        mlflow.log_param("num_features",100)
        mlflow.log_param("test_size",0.2)
        
        logging.info("Initilize the Logistic regerssion model ")
        model = LogisticRegression(max_iter=1000)
        
        logging.info("fit the model ")
        model.fit(X_train,y_train)
        
        logging.info("Model training completed ")
        
        logging.info("log the model params ")
        mlflow.log_param("model", "Logistic Regression")
        
        logging.info("starting prediction on model ")
        y_pred = model.predict(X_test)
        
        logging.info("calculate performcance metrices")
        accuracy = accuracy_score(y_test,y_pred)
        precision = precision_score(y_test,y_pred)
        recall = recall_score(y_test,y_pred)
        f1 = f1_score(y_test,y_pred)
        
        logging.info("logging and printing above performance metrics ")
        mlflow.log_metric("accuracy" ,accuracy )
        mlflow.log_metric("precision",precision)
        mlflow.log_metric("recall",recall)
        mlflow.log_metric("f1",f1)   
        
        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")
        
        logging.info("saving the model ")
        mlflow.sklearn.log_model(sk_model=model,name="logistic_regression_model")

        
        logging.info(f"total time taken in execution {time.time() - start_time}")
    except Exception as e:
        raise e

2026-02-08 19:37:42,788 - INFO - starting mlflow run   
2026-02-08 19:37:54,893 - WARNING - Retrying (Retry(total=6, connect=6, read=7, redirect=7, status=7)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A243076F90>: Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /chauhan7gaurav/mlops_capstone.mlflow/api/2.0/mlflow/runs/create
2026-02-08 19:38:04,680 - INFO - logging model parameters 
2026-02-08 19:38:05,796 - INFO - Initilize the Logistic regerssion model 
2026-02-08 19:38:05,799 - INFO - fit the model 
2026-02-08 19:38:05,915 - INFO - Model training completed 
2026-02-08 19:38:05,919 - INFO - log the model params 
2026-02-08 19:38:06,263 - INFO - starting prediction on model 
2026-02-08 19:38:06,266 - INFO - calculate performcance metrices
2026-02-08 19:38:06,299 - INFO - logging and printing above performance metrics 
2026-02-08 19:38:10,882 - INFO - Accuracy: 0.7
2026-02-08 19:38:10,885 - INFO - Preci

🏃 View run rumbling-dove-990 at: https://dagshub.com/chauhan7gaurav/mlops_capstone.mlflow/#/experiments/0/runs/0a263ec9d5484114b0416010da5095b2
🧪 View experiment at: https://dagshub.com/chauhan7gaurav/mlops_capstone.mlflow/#/experiments/0
